# Init

In [1]:
import sys
import os
import json
import time

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data

In [2]:
system_prompt = ""

def make_prompt(conversation_turn: dict) -> str:
    database_relationship = """#### **库表关系**
```text
|库名中文|库名英文|表英文|表中文|表描述|
| 上市公司基本资料| AStockBasicInfoDB| LC_StockArchives| 公司概况| 收录上市公司的基本情况，包括：联系方式、注册信息、中介机构、行业和产品、公司证券品种及背景资料等内容。 |
| 上市公司基本资料| AStockBasicInfoDB| LC_NameChange| 公司名称更改状况| 收录公司名称历次变更情况，包括：中英文名称、中英文缩写名称、更改日期等内容。|
| 上市公司基本资料| AStockBasicInfoDB| LC_Business| 公司经营范围与行业变更| 1.收录上市公司、发债公司的经营范围（包括主营和兼营）以及涉足行业情况。 2.信息来源：公开转让说明书、董事会决议、定报、临时公告等。|
| 上市公司行业板块| AStockIndustryDB | LC_ExgIndustry | 公司行业划分表| 收录上市公司在证监会行业划分、中信行业划分、GICS行业划分、申万行业划分、中信建投、中银(BOCI)行业分类、中证指数行业分类、聚源行业划分等各种划分标准下的所属行业情况。|
| 上市公司行业板块| AStockIndustryDB | LC_ExgIndChange| 公司行业变更表| 内容说明： 本表记录上市公司从上市至今，由于主营业务变更导致的所属行业变化情况，采用同一行业分类标准，对其历史变更进行人为追溯，以便投资者进行公司数据回测，或开展行业估值、财务等数据的计算。 本表对公司所属行业的变更情况尽量参照原行业分类发布公司的披露数据，并对其新旧分类标准的不同之处加以判断，结合公司实际业务的变化，逐一进行人工比对，用最新的行业标准反映公司历史上的行业变更情况。 数据范围：A股上市公司 信息来源：公司公告、聚源整理。|
| 上市公司行业板块| AStockIndustryDB | LC_IndustryValuation| 行业估值指标| 内容说明：本表记录不同行业标准下的的衍生指标，包括行业静态市盈率、滚动市盈率、市净率、股息率等指标。 数据范围：2014-01-01至今 信息来源：聚源计算 |
| 上市公司行业板块| AStockIndustryDB | LC_IndFinIndicators| 行业财务指标表| 1.内容说明：本表存储行业衍生指标相关数据，反映不同行业分类标准下，各行业的成长能力、偿债能力、盈利能力和现金获取能力等。本表数据多采用整体法进行计算（如计算增长率时，采用（行业内所有公司的当期总值-上期总值）/上期总值，而非行业内公司增长率的算术平均值），且部分比例类指标对金融类公司不适用（流动比例、速动比例、毛利率等），该类指标未计算金融类公司。 2.数据范围：A股财报、业绩快报、股本结构、分红等数据，2014年至今。 3.信息来源：公告披露，聚源计算。|
| 上市公司行业板块| AStockIndustryDB | LC_COConcept| 概念所属公司表| 记录A股上市公司所属概念信息。|
| 上市公司行业板块| AStockIndustryDB | LC_ConceptList | 概念板块常量表| 记录A股市场中热点概念的相关信息|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_SuppCustDetail| 公司供应商与客户| 1.内容说明：收录A股上市公司的主要供应商、客户清单，以及交易标的、交易金额等信息。 2.数据范围：2015年至今 3.信息来源：招股说明书、定报 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SHTypeClassifi| 股东类型分类表| 本表记录聚源股东类型分类数据|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_MainSHListNew| 股东名单(新)| 1.收录公司主要股东构成及持股数量比例、持股性质等明细资料，包括发行前和上市后的历次变动记录。 2.数据范围：1992-06-30至今 3.信息来源：招股说明书、上市公告书、定报、临时公告等。 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SHNumber| 股东户数| 1.反映公司全体股东、A股股东、B股东、H股东、CDR股东的持股情况及其历史变动情况等。 2.指标计算公式： 1)户均持股比例=((股本/股东总户数)/股本)*100%（公式中分子分母描述同一股票类型） 2)相对上一期报告期户均持股比例变化=本报告期户均持股比例－上一报告期户均持股比例 3)户均持股数季度增长率=(本季度户均持股数量/上一季度户均持股数量-1)*100% 4)户均持股比例季度增长率=(本季度户均持股比例/上一季度户均持股比例-1)*100% 5)户均持股数半年增长率=(本报告期户均持股数量/前推两季度户均持股数量-1)*100%6)户均持股比例半年增长率 = (本报告期户均持股比例/前推两个季度户均持股比例-1)*100% 2.数据范围：1991-1-1至今 3.信息来源：招股说明书、上市公告书、定报、临时公告、深交所互动易、上证e互动等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_Mshareholder| 大股东介绍| 1.收录上市公司及发债企业大股东的基本资料，包括直接持股和间接持股，以及持股比例、背景介绍等内容。 2.数据范围：2004-12-31至今 3.信息来源：募集说明书、招股说明书、定报、临时公告等|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ActualController| 公司实际控制人| 1.收录根据上市公司在招投说明书、定期报告、及临时公告中披露的实际控制人结构图判断的上市公司实际控制人信息。2.目前只处理实际控制人有变动的数据，下期和本期相比如无变化，则不做处理。 3.数据范围：2004-12-31至今 4.信息来源：招股说明书、上市公告书、定报、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareStru| 公司股本结构变动| 1.收录上市公司股本结构历史变动情况。其中：标注“披露”的字段为公司公告原始披露，标注“计算”的字段为聚源依据股权登记日，并且考虑高管股锁定的实际情况计算所得的股本结构。 2.数据范围：1990-12-10至今 3.信息来源：招股说明书、上市公告书、定报、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_StockHoldingSt| 股东持股统计| 1.收录报告期末，各类机构投资者对每只股票的持仓情况，以及前十大（无限售条件）股东合计持股情况等。 2.机构持股统计中，基金持股综合考虑了上市公司披露的十大股东数据以及基金报告中披露的基金持股数据；机构持股合计包含上市公司披露的股东持股以及在同一截止时点上基金披露的所持股票数据。 3.计算公式： 1)机构持有无限售流通股数量=机构持有无限售流通A股之和2)机构持有无限售流通股比例=(机构持有无限售流通股数量/无限售A股)*100%3)机构持有A股数量=机构持有A股之和4)机构持有A股比例=(机构持有A股数量/A股总数)*100%5)机构持有股票数量=机构持有股票之和6)机构持有股票比例=(机构持有股票数量/总股本)*100% 4.数据范围：1992年至今 5.信息来源：招股说明书、上市公告书、定报、临时公告等|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareTransfer| 股东股权变动| 1.收录公司股东股权转让、二级市场买卖、股权拍卖、大宗交易、股东重组等引起股东股权变动方面的明细资料，并包含与股权分置改革相关的股东增持、减持等信息。 2.数据范围：1996-01-26至今 3.信息来源：上交所和深交所大宗交易公开信息、临时公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareFP | 股东股权冻结和质押| 1.收录股东股权被冻结和质押及进展情况，包括被冻结质押股东、被接受股权质押方、涉及股数以及冻结质押期限起始和截止日等内容。 2.数据范围：1999-09-30至今 3.信息来源：股权质押公告、股权冻结公告、解除质押冻结公告等。 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ShareFPSta| 股东股权冻结和质押统计| 1.收录股东股权的质押冻结统计数据，包括股东股权累计冻结质押股数、累计占冻结质押方持股数比例和累计占总股本比例等情况。 2.指标计算公式： 1)累计占冻结质押方持股数比例=股东累计冻结质押股数(股)/股东持股数 2)累计占总股本比例 =股东累计冻结质押股数(股)/公司总股本 3)累计占总股本比例(计算) =股东累计冻结质押股数(股)/公司总股本 3.数据范围：2006-05-15至今 4.信息来源：股权质押公告、股权冻结公告、解除质押冻结公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_Buyback | 股份回购| 1.介绍上市公司(包含科创板)发生股份回购的相关方案信息，包括股份类别、首次信息发布日期、回购协议签署日、股份被回购方、回购数量上限与下限、回购价格上限与下限、回购期限起始与截止日等内容。 2.数据范围：1994-06-23至今 3.信息来源：回购公告、董事会公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_BuybackAttach| 股份回购关联表| 1.补充上市公司(包含科创板)发生股份回购的相关信息，包括本次回购数量、累计回购数量、本次回购资金和累计回购数量等内容。 2.数据范围：1994-09-27至今 3.信息来源：回购公告、董事会公告等。|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_LegalDistribution| 法人配售与战略投资者 | 1.收录公司首次发行、增发新股、发行可转债过程中采用网下配售方式过程中，获得配售的企业、基金明细。 2.数据范围：1994-04-23至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_NationalStockHoldSt| A股国家队持股统计 | 1.内容说明：本表记录股市国家队成员持有A股的相关信息，包含：持有A股总数，占总股本比例，持有A股数量增减，持有A股数量增减幅度等。 2.数据范围：2003-01-01至今 3.信息来源：聚源 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| CS_ForeignHoldingSt| 外资持股统计| 内容说明：境外投资者持股统计，包含持股总数、持股比例，境外投资者指QFII/RQFII/深股通/全球存托凭证跨境转换机构/全球存托凭证存托人。 数据范围：2007年至今 信息来源：深交所、上交所|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_AShareSeasonedNewIssue | A股增发| 1.收录A股增发A股、B股增发A股、H股增发A股等的明细情况，包括历次增发预案、进程日期、预案有效期、发行属性、发行价区间、发行量区间、发行日期、上网发行情况、网下配售申购情况和募集资金与费用等内容。 2.数据范围：1991-08-17至今|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_ASharePlacement | A股配股| 1.收录A股历次配股预案及实施进展明细，包括预案有效期、配股价格区间、配股说明书、募集资金和配股交款日等内容。 2.数据范围：1991-03-06至今 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_Dividend| 公司分红| 1.该表包括上市公司历次分红预案及实施进展，以及下年分配次数、方式等，以分红事件为维度，一次分红做一条记录。 2.数据范围：证券上市起-至今 3.信息来源：上市公司公告 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_CapitalInvest| 资金投向说明| 1.公司自有资金、通过发行新股、增发新股、配股、发行可转债、发行企业债等方式所得募集资金的项目投资情况以及运用进展和改投状况。 2.数据范围：1988-12-01至今 3.信息来源：董事会公告、招股意向书、招股说明书等 |
| 上市公司股票行情| AStockMarketQuotesDB | CS_StockCapFlowIndex| 境内股票交易资金流向指标 | 内容说明： 1、收录深沪京交易所正常交易的股票在每个交易日基于不同成交金额区间及成交时间区间主动及含主动被动交易的累计流入流出金额、量等信息衍生计算的统计类指标 2、数据提供范围说明 2023-10-09 及以后提供完整全盘、开盘、尾盘主买主卖及含主动被动数据 2022-11-15~2023-09-28 仅提供全盘主买主卖及含主动被动资金流向数据 2016-11-29~2022-11-14 仅提供全盘含主动被动资金流向数据 数据范围：2016-11-29至今 信息来源：基于交易所行情数据衍生计算|
| 上市公司股票行情| AStockMarketQuotesDB | CS_TurnoverVolTecIndex| 境内股票成交量技术指标| 内容说明：收录境内股票上市之日起基于日、周、月、季、半年、年K线行情衍生计算的成交量技术指标 数据范围：股票上市起-至今 信息来源：基于沪深京交易所及股转系统行情数据衍生计算 |
| 上市公司股票行情| AStockMarketQuotesDB | CS_StockPatterns| 股票技术形态表| 内容说明：收录股票从最近一个交易日往前追溯一段时期的行情表现和技术形态表现，包括近1周、近1月、近3月、近半年、近1年、上市以来的表现情况，以及连涨跌天数、连续放量缩量天数、向上向下有效突破均线、N天M板、均线多空头排列看涨看跌等技术形态指标。本表覆盖的证券品种有A股、B股、中国存托凭证(CDR), 覆盖的上市标志有主板、三板、创业板、科创板。数据范围：股票上市或挂牌起-至今 信息来源：基于沪深京交易所及股转系统行情数据衍生计算|
| 上市公司股票行情| AStockMarketQuotesDB | QT_DailyQuote| 日行情表| 1.收录股票、债券（不包含银行间交易的债券）、基金、指数每个交易日收盘行情数据，包括昨收盘、今开盘、最高价、最低价、收盘价、成交量、成交金额、成交笔数等行情指标。 2.数据范围：证券上市起-至今 3.信息来源：上交所/深交所/北交所每日行情收盘文件|
| 上市公司股票行情| AStockMarketQuotesDB | QT_StockPerformance| 股票行情表现(新)| 1.内容说明： 收录股票从最近一个交易日往前追溯一段时期的行情表现信息，包括近1周、1周以来、近1月、1月以来、近3月、近半年、近1年、今年以来、上市以来的表现情况，以及β、α、波动率、夏普比率等风险指标，本表包含停牌数据。 计算方法： 1)区间成交金额=∑区间每个交易日成交金额2)区间成交量=∑区间每个交易日成交量3)区间涨跌幅=(区间内最新复权收盘价/区间首日复权昨收盘－1)*1004)区间振幅=(区间最高复权价－区间最低复权家价)/区间首日复权昨收盘*1005)区间换手率=区间每一天换手率的合计值6) 区间成交均价=区间成交金额之和/区间成交量之和（考虑了区间有除权的情况）7) 区间日均成交金额=区间成交金额之和/区间实际交易天数8) 区间日均换手率=区间每日换手率之和/区间实际交易天数 2.数据范围：股票上市起-至今 3.信息来源：基于沪深京交易所行情数据衍生计算 |
| 上市公司股票行情| AStockMarketQuotesDB | LC_SuspendResumption| 停牌复牌表| 1.收录上市公司/基金/债券停牌复牌信息，如停牌日期、停牌时间、停牌原因、停牌事项说明、停牌期限、复牌日期、复牌时间、复牌事项说明等，包括盘中临时停牌。 2.数据范围：2008.04-至今 2.信息来源：上海证券交易所、深圳证券交易所、北京证券交易所|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_BalanceSheetAll | 资产负债表_新会计准则| 1.反映企业依据2007年新会计准则在年报、中报、季报中披露的资产负债表数据；并依据新旧会计准则的科目对应关系，收录主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1989-12-31至今 7.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_IncomeStatementAll | 利润分配表_新会计准则| 1.反映企业依据2007年新会计准则在在年报、中报、季报中披露的利润表数据；并依据新旧会计准则的科目对应关系，收录了主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1989-12-31至今 7.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_CashFlowStatementAll| 现金流量表_新会计准则| 1.反映企业依据2007年新会计准则在年报、中报、季报中披露的现金流量表数据；并依据新旧会计准则的科目对应关系，收录了主要科目的历史对应数据。 2.收录同一公司在报告期末的四种财务报告，即未调整的合并报表、未调整的母公司报表、调整后的合并报表以及调整后的母公司报表。 3.若某个报告期的数据有多次调整，则该表展示历次调整数据。 4.该表中各财务科目的单位均为人民币元。 5.带“##”的特殊项目为单个公司披露的非标准化的科目，对应的“特殊字段说明”字段将对其作出说明；带“##”的调整项目是为了让报表的各个小项借贷平衡而设置的，便于客户对报表的遗漏和差错进行判断。 6.数据范围：1998-06-30至今 7.信息来源：招股说明书、定报、审计报告等 |
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_IntAssetsDetail | 公司研发投入与产出| 1.内容说明：收录上市公司研发投入相关数据，主要包括研发费用投入总额、占比，研发人员构成、占比等信息。 2.数据范围：2014年至今 3.信息来源：定期报告|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_MainOperIncome| 公司主营业务构成| 1收录公司主营业务的收入来源、成本构成；主营业务收入、成本和利润与上年同期的对比较。 2.数据范围：1998-12-31至今 3.信息来源：招股说明书、定报、审计报告等|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_OperatingStatus | 公司经营情况述评| 1.收录公司管理层对季度、半年度、年度经营情况的自我评价，以及其后期发展计划和预测，本表涵盖了公司招股以来的历次纪录。 2.数据范围：1997-12-31至今 3.信息来源：定期报告|
| 上市公司财务指标/财务报表/融资与分红 | AStockFinanceDB| LC_AuditOpinion| 公司历年审计意见| 1.收录中介机构对公司季度、半年度、年度经营情况的评价，区分审计单位、审计意见类型，本表涵盖了公司招股以来的历次纪录。 2.数据范围：1990-12-31至今 3.信息来源：定期报告、审计报告等|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_Staff| 公司职工构成| 1.从技术职称、专业、文化程度、年龄等几个方面介绍公司职工构成情况。 2.数据范围：1999-12-31至今 3.信息来源：定期报告、招股说明书等|
| 上市公司产品供销/人力资源 | AStockOperationsDB| LC_RewardStat| 公司管理层报酬统计| 1.按报告期统计管理层的报酬情况，包括报酬总额、前三名董事报酬、前三名高管报酬、报酬区间统计分析等。 2.数据范围：2001-12-31至今 3.信息来源：定期报告、招股说明书等|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Warrant | 公司担保明细| 1.收录上市公司公告中披露的担保等重大事项，包括时间内容、最新进展、事件主体/交易对象名称、企业编号、与上市公司关联关系、担保原因等指标。 2.数据范围：2001年-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Credit| 公司借贷明细| 1.收录上市公司公告中披露的公司借贷等重大事项描述，包括时间内容、时间主体、交易对象名称、借贷金额、还款金额、借贷利率、借贷期限等指标。 2.数据范围：2001年-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_SuitArbitration | 公司诉讼仲裁明细| 1.公司诉讼仲裁等重大事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、诉讼仲裁金额、原告及与上市公司关联关系、被告及与上市公司关联关系、仲裁状态等指标。 2.数据范围：2001-至今 3.信息来源：上市公司临时公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_EntrustInv| 重大事项委托理财| 1.公司委托贷款等重大事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、涉及金额、委托期限、委托起始日、委托截止日等指标。 2.数据范围：2001-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_Regroup | 公司资产重组明细| 1.公司资产重组，如资产出售与转让、资产置换、债权债务重组等重大事项描述说明。 2.数据范围：2001-至今 3.信息来源：上市公司公告|
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_MajorContract| 公司重大经营合同明细 | 1.本表存放公司重大经营合同的事项，包括事件主体/交易对象名称、企业编号、与上市公司关联关系、合同标的、合同获得方式、涉及金额、合同起始日、合同截止日、合同期限等指标。 2.数据范围：2012-至今 3.信息来源：上市公司公告 |
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_InvestorRa| 投资者关系活动| 1.收录各调研机构对上市公司调研的详情，包括调研日期、参与单位、调研人员、调研主要内容等信息。 2.数据范围：2012-至今 3.信息来源：巨潮，上交所互动易和深交所互动易 |
| 上市公司公告资讯/重大事项 | AStockEventsDB| LC_InvestorDetail| 投资者关系活动调研明细| 1、收录参与上市公司调研活动的调研机构明细数据，包括调研单位、调研人员等指标。 2、数据范围：2016-至今 3、信息来源：交易所，上交所互动易和深交所互动易 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ESOP| 员工持股计划| 1.主要记录员工持股计划当期的情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、股票来源、股票规模、实施是否分期、存续期、锁定期等一些情况。 2.数据范围：2014.6-至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_ESOPSummary | 员工持股计划概况| 1.本表主要记录员工持股计划总体情况：包括相关日期、事件进程、事件说明、资金来源、资金总额、股票来源、股票规模等一些情况。对于一些分期实施的员工持股计划，本表记录总体计划的情况。 2.数据范围：2014.6-至今 3.信息来源：上市公司公告|
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_TransferPlan| 股东增减持计划表| 1.内容说明：收录上市公司(包含科创板)股东增持计划、减持计划、被动减持计划、不减持类别指标。 2.数据范围：2005-至今 3.信息来源：上市公司公告 |
| 上市公司股东与股本/公司治理| AStockShareholderDB| LC_SMAttendInfo| 股东大会出席信息| 1.收录股东大会召开时间，地点，类别；投票方式；见证律师事务所及经办律师；全体股东出席情况；非流通股东出席情况；流通股东出席情况。 2.数据范围：1999-1-28至今|
| 港股数据库 | HKStockDB | HK_EmployeeChange| 港股公司员工数量变动表| 1.记录港股公司员工数量的变动历史记录数据，包括信息发布日期、信息来源、生效日期、变更前员工数量、变更后员工数量等。2.数据范围：2001年至今。 3.信息来源：港交所。 |
| 港股数据库 | HKStockDB | HK_StockArchives| 港股公司概况| 1.收录港股上市公司的基础信息，包括名称、成立日期、注册地点、注册资本、公司业务、所属行业分类、主席、公司秘书、联系方式等信息。 2.信息来源：港交所等。 |
| 港股数据库 | HKStockDB | CS_HKStockPerformance | 港股行情表现| 1.内容说明： 收录股票从最近一个交易日往前追溯一段时期的行情表现信息，包括近1周、1周以来、近1月、1月以来、近3月、近半年、近1年、今年以来、上市以来的表现情况，本表包含停牌数据。 2.数据范围：2005年至今。 3.数据来源：根据港交所披露数据聚源衍生计算。|
| 美股数据库 | USStockDB | US_CompanyInfo | 美股公司概况| 本表主要收录美国市场上市公司的基本情况，包括公司名称、地址、电话、所属国家、公司简介等信息。|
| 美股数据库 | USStockDB | US_DailyQuote| 美股日行情| 1.本表收录美国市场证券的日收盘行情 2.数据范围：2000年2月至今|
| 公募基金数据库 | PublicFundDB| MF_FundArchives| 公募基金概况| 1.本表记录了基金基本情况，包括基金规模、成立日期、投资类型、管理人、托管人、存续期、历史简介等。2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网披露的产品说明书。|
| 公募基金数据库 | PublicFundDB| MF_FundProdName| 公募基金产品名称| 1.本表记录基金的交易所披露简称、集中申购简称、ETF申购赎回简称等基金相关的名称类信息。 2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网披露的产品说明书。其中，4-证监会简称处理的是资本市场电子化信息披露平台-公募基金净值日报的简称；6-公告披露简称处理的是基金产品资料概要和定报披露的简称；8-基金全称处理的是发售公告或是资本市场电子化信息披露平台-基金概况的全称，是将基金的多个份额合并的基金全称。|
| 公募基金数据库 | PublicFundDB| MF_InvestAdvisorOutline| 公募基金管理人概况| 1.本表记录了基金管理人的基本情况介绍，包括成立日期、注册资本、法人代表、联系方式、背景简介等。 2.历史数据：1998年3月起-至今。 3.信息来源：基金公司官网。 |
| 公募基金数据库 | PublicFundDB| MF_Dividend| 公募基金分红| 1.本表记录基金单次分红信息，包括分红比例、登记日、除息日等信息，以及聚源根据相关数据计算的累计分红金额、累计分红次数等数据。 2.历史数据：1998年12月起-至今。 3.信息来源：基金公司官网披露的相关临时公告。|
| 诚信数据库 | CreditDB| LC_ViolatiParty| 违规当事人处罚| 1.该表以事件+当事人+处罚为维度，记录单个事件下单个当事人的每一个处罚，包括当事人及其性质、当事人编码、开始日期、截止日期、违规类型、关联关系、关联上市公司、处罚机构编码、处罚机构、涉及金额、处罚说明等指标。 2.数据范围：2014年-至今 3.信息来源：交易所、上市公司公告、证监会等|
| 指数数据库 | IndexDB| LC_IndexBasicInfo| 指数基本情况| 1.收录了市场上主要指数的基本情况，包括指数类别、成份证券类别、发布机构、发布日期、基期基点、指数发布的币种等信息。 2.数据源：中证指数有限公司、上海证券交易所、深圳证券交易所、中央国债登记结算有限责任公司、申银万国研究所、标普道琼斯指数公司等|
| 指数数据库 | IndexDB| LC_IndexComponent| 指数成份| 1.收录了市场上主要指数的成份证券构成情况，包括成份证券的市场代码、入选日期、删除日期以及成份标志等信息。 2.该表仅收录主指数成份信息，不收录与主指数关系（Relationship）为“1-币种不同，2-分红规则不同，3-分红规则和币种都不同，4-税后分红”的衍生指数的信息。 3.历史数据：1990年12月至今 4.数据源：中证指数有限公司、上海证券交易所、深圳证券交易所、申银万国研究所等 |
| 机构数据库 | InstitutionDB | LC_InstiArchive| 机构基本资料| 1.收录市场上重要机构的基本资料情况，如证券公司、信托公司、保险公司等；包含机构名称、机构信息、联系方式、机构背景等信息. 2.数据源：国家企业信用信息公示系统等. |
| 常量库 | ConstantDB| SecuMain| 证券主表| 本表收录单个证券品种（股票、基金、债券）的代码、简称、上市交易所等基础信息。|
| 常量库 | ConstantDB| HK_SecuMain| 港股证券主表| 本表收录港股单个证券品种的简称、上市交易所等基础信息。|
| 常量库 | ConstantDB| CT_SystemConst | 系统常量表| 本表收录数据库中各种常量值的具体分类和常量名称描述。|
| 常量库 | ConstantDB| QT_TradingDayNew| 交易日表(新)| 本表收录各个市场的交易日信息，包括每个日期是否是交易日，是否周、月、季、年最后一个交易日|
| 常量库 | ConstantDB| LC_AreaCode| 国家城市代码表| 本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。 |
| 机构数据库 | InstitutionDB | PS_EventStru| 事件体系指引表| 收录聚源最新制定的事件分类体系。|
| 常量库 | ConstantDB| US_SecuMain| 美股证券主表| 本表收录美国等境外市场单个证券品种的简称、上市交易所等基础信息。 |
| 机构数据库 | InstitutionDB | PS_NewsSecurity| 证券舆情表| 收录了全网披露的舆情信息涉及的相关证券，对对应的事件信息，并对相应的事件的正负面情感及情感重要性进行等级划分。|
```"""

    task_description = """\n
#### **任务描述**
请为对话内容中的每一轮对话（turn）推断其答案需要查询的数据库和表，并以 JSON 结构返回结果。具体步骤如下：
1. **推理数据源**：根据对话轮次中的每个问题内容，推理回答该问题所需的数据来源（包括数据库和表）。在 `data_source_reasoning` 字段中详细写出推理过程。
2. **确定数据源**：在 `data_source` 字段中明确给出需要查询的数据库和表的具体名称。

返回结果的 JSON 结构如下：
```json
{
    "tid": "对话轮次唯一标识",
    "team": [
        {
            "id": "问题的唯一标识",
            "question": "问题内容",
            "data_source_reasoning": "推理过程...",
            "data_source": [
                {"database": "数据库英文名", "table": "表英文名"}
            ]
        },
        ...
    ]
}
```"""

    few_shots = """\n
#### **示例（Shots）**

**示例 1**
```json
{
    "tid": "tttt----1",
    "team": [
        {
            "id": "tttt----1----1-1-1",
            "question": "600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？",
            "data_source_reasoning": "该问题需要查询上市公司的基本信息，包括公司全称、A股简称、法人等，这些信息通常记录在上市公司基本资料数据库的公司概况表中。",
            "data_source": [
                {"database": "AStockBasicInfoDB", "table": "LC_StockArchives"}
            ]
        },
        {
            "id": "tttt----1----1-1-2",
            "question": "该公司实控人是否发生改变？如果发生变化，什么时候变成了谁？是哪国人？是否有永久境外居留权？（回答时间用XXXX-XX-XX）",
            "data_source_reasoning": "问题涉及实控人的变更情况，包括变更时间和背景信息，这些内容可以在上市公司股东与股本相关的公司实际控制人表中找到。",
            "data_source": [
                {"database": "AStockShareholderDB", "table": "LC_ActualController"}
            ]
        },
        {
            "id": "tttt----1----1-1-3",
            "question": "在实控人发生变化的当年股权发生了几次转让？",
            "data_source_reasoning": "问题涉及实控人变更当年的股权转让情况，这些信息可以从股东股权变动的记录中查询到。",
            "data_source": [
                {"database": "AStockShareholderDB", "table": "LC_ShareTransfer"}
            ]
        }
    ]
}
```

**示例 2**
```json
{
    "tid": "tttt----2",
    "team": [
        {
            "id": "tttt----2----2-1-1",
            "question": "今天是2021年12月24日，创近半年新高的股票有几只？",
            "data_source_reasoning": "此问题需要统计近半年内股票的最高点信息，这属于股票技术形态的分析内容，可以通过市场行情数据库的股票技术形态表获取。",
            "data_source": [
                {"database": "AStockMarketQuotesDB", "table": "CS_StockPatterns"}
            ]
        },
        {
            "id": "tttt----2----2-1-2",
            "question": "哪些股票股价大于75，且同时当天创一年新高的是？",
            "data_source_reasoning": "此问题需要获取每日收盘数据，并结合股票技术形态分析是否创一年新高。涉及到的表包括每日行情表和股票技术形态表。",
            "data_source": [
                {"database": "AStockMarketQuotesDB", "table": "QT_DailyQuote"},
                {"database": "AStockMarketQuotesDB", "table": "CS_StockPatterns"}
            ]
        },
        {
            "id": "tttt----2----2-1-3",
            "question": "以上股票连续两日（今日与昨日）满足上述要求的是？",
            "data_source_reasoning": "此问题需要获取每日行情数据，并对比分析连续两日的数据，判断是否满足条件。涉及每日行情表和技术形态表。",
            "data_source": [
                {"database": "AStockMarketQuotesDB", "table": "QT_DailyQuote"},
                {"database": "AStockMarketQuotesDB", "table": "CS_StockPatterns"}
            ]
        }
    ]
}
```

**示例 3**
```json
{
    "tid": "tttt----3",
    "team": [
        {
            "id": "tttt----3----3-2-1",
            "question": "2019年度（合并）常熟银行的职工总数有多少人？",
            "data_source_reasoning": "此问题需要查询公司员工总数的信息，可以从产品供销与人力资源数据库的职工构成表获取。",
            "data_source": [
                {"database": "AStockOperationsDB", "table": "LC_Staff"}
            ]
        },
        {
            "id": "tttt----3----3-2-2",
            "question": "该公司披露的硕士或研究生学历（及以上）的有多少人？",
            "data_source_reasoning": "问题需要获取公司员工的学历构成信息，可以从产品供销与人力资源数据库的职工构成表中找到。",
            "data_source": [
                {"database": "AStockOperationsDB", "table": "LC_Staff"}
            ]
        },
        {
            "id": "tttt----3----3-2-3",
            "question": "20201月1日至年底退休了多少人？",
            "data_source_reasoning": "问题需要获取员工退休情况的信息，这些数据存储在职工构成表中。",
            "data_source": [
                {"database": "AStockOperationsDB", "table": "LC_Staff"}
            ]
        }
    ]
}
```"""

    tips = """\n
#### **注意事项**
1. 每个示例包含完整的对话轮次（一个 turn），其中所有问题的推理和数据源完整描述。
2. 确保 JSON 结构清晰，方便直接处理。"""

    current_turn = """\n
#### **对话内容**
```json
""" + json.dumps(conversation_turn, ensure_ascii=False, indent=4) + "\n```"

    prompt = database_relationship + current_turn + task_description + few_shots + tips

    return prompt

In [3]:
question_path = os.path.join(cwd, 'data' + os.sep + 'question.json')

questions = parse_data.read_json(question_path)

# GLM-4-Plus

## Test

In [ ]:
query = make_prompt(questions[12])

history = []

start_time = time.time()
message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.1, top_p=1, response_format='text')
end_time = time.time()

execution_time = end_time - start_time
usage = chat.get_token_usage(message, True)
content = chat.get_content(message, True)
history = chat.build_history(history, message=message)

In [ ]:
t = json.loads(content.strip('`json'))
t['token_usage'] = {}
t['token_usage']['stage_1'] = usage
t['time_usage'] = {}
t['time_usage']['stage_1'] = f"{execution_time:.2f}s"
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-answer-test.json')
parse_data.write_json(t, saved_path)

## ALL

In [6]:
answers = []

for question in tqdm(questions[:]):
    
    query = make_prompt(question)

    history = []

    start_time = time.time()
    message = chat.create_message(query, history=history, system_prompt=system_prompt, temperature=0.1, top_p=1, response_format='text')
    end_time = time.time()

    execution_time = end_time - start_time
    usage = chat.get_token_usage(message, False)
    content = chat.get_content(message, False)

    content = json.loads(content.strip('`json'))
    content['token_usage'] = {}
    content['token_usage']['stage_1'] = usage
    content['time_usage'] = {}
    content['time_usage']['stage_1'] = f"{execution_time:.2f}s"

    answers.append(content)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-answer.json')
parse_data.write_json(answers, saved_path)

100%|██████████| 101/101 [25:50<00:00, 15.35s/it]


# Deepseek-chat

In [33]:
model = 'deepseek_241206'

## Test

In [31]:
from openai import OpenAI

query = make_prompt(questions[12])

client = OpenAI(api_key= deepseek_api, base_url="https://api.deepseek.com")

start_time = time.time()
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": query},
    ],
    stream=False,
    top_p=0.7,
    temperature=0.9
)
end_time = time.time()

response = json.loads(response.to_json())
content = response['choices'][0]['message']['content']

content = content.strip('`json')
usage = response['usage']
execution_time = end_time - start_time

In [35]:
t = json.loads(content)
t['token_usage'] = {}
t['token_usage']['stage_1'] = usage
t['time_usage'] = {}
t['time_usage']['stage_1'] = f"{execution_time:.2f}s"
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-answer-test.json')
parse_data.write_json(t, saved_path)

## ALL

In [37]:
answers = []

for question in tqdm(questions[:]):
    
    query = make_prompt(question)

    start_time = time.time()
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": query},
        ],
        stream=False,
        top_p=0.7,
        temperature=0.9
    )
    end_time = time.time()

    response = json.loads(response.to_json())
    content = response['choices'][0]['message']['content']

    content = content.strip('`json')
    usage = response['usage']
    execution_time = end_time - start_time

    content = json.loads(content.strip('`json'))
    content['token_usage'] = {}
    content['token_usage']['stage_1'] = usage
    content['time_usage'] = {}
    content['time_usage']['stage_1'] = f"{execution_time:.2f}s"

    answers.append(content)

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-answer.json')
parse_data.write_json(answers, saved_path)

100%|██████████| 101/101 [36:30<00:00, 21.69s/it]


# Claude-3-5-sonnet-20241022

In [5]:
model = "claude_3_5_sonnet_20241022"

## Test

In [59]:
import anthropic


query = make_prompt(questions[12]) + '\n返回json。'

client = anthropic.Anthropic(api_key=claude_api)

start_time = time.time()
response = client.beta.prompt_caching.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=4000,
    temperature=0.9,
    top_p=0.1,
    messages=[{"role": "user", "content": query}],
)
end_time = time.time()

response = json.loads(response.to_json())
content = response['content'][0]['text']

usage = response['usage']
execution_time = end_time - start_time

In [66]:
t = json.loads(content)
t['token_usage'] = {}
t['token_usage']['stage_1'] = usage
t['time_usage'] = {}
t['time_usage']['stage_1'] = f"{execution_time:.2f}s"
t = [t]

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-answer-test.json')
parse_data.write_json(t, saved_path)

## ALL

In [68]:
answers = []

for question in tqdm(questions[:]):
    
    query = make_prompt(question) + '\n返回json。'

    start_time = time.time()
    response = client.beta.prompt_caching.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=4000,
        temperature=0.9,
        top_p=0.7,
        messages=[{"role": "user", "content": query}],
    )
    end_time = time.time()

    response = json.loads(response.to_json())
    content = response['content'][0]['text']

    usage = response['usage']
    execution_time = end_time - start_time

    try:
        content = json.loads(content)
        content['token_usage'] = {}
        content['token_usage']['stage_1'] = usage
        content['time_usage'] = {}
        content['time_usage']['stage_1'] = f"{execution_time:.2f}s"

        answers.append(content)
    except:
        print('-------')
        print(question)
        print('-------')

saved_path = os.path.join(cwd, 'answer_tmp' + os.sep + f'stage_1-{model}-answer.json')
parse_data.write_json(answers, saved_path)

 16%|█▌        | 16/101 [02:30<12:03,  8.51s/it]

-------
{'tid': 'tttt----16', 'team': [{'id': 'tttt----16----16-2-1', 'question': '2021-12-31报告期内，查询总资产最高的公司是?'}, {'id': 'tttt----16----16-2-2', 'question': '经营活动产生的现金流量净额最高的公司是？'}, {'id': 'tttt----16----16-2-3', 'question': '这两个公司净利润差额是？'}]}
-------


 45%|████▍     | 45/101 [07:13<09:58, 10.68s/it]

-------
{'tid': 'tttt----45', 'team': [{'id': 'tttt----45----28-1-1', 'question': '易方达基金管理有限公司在19年成立了多少支基金？'}, {'id': 'tttt----45----28-1-2', 'question': '哪支基金的规模最大？'}, {'id': 'tttt----45----28-1-3', 'question': '这支基金20年最后一次分红派现比例多少钱？'}]}
-------


100%|██████████| 101/101 [17:39<00:00, 10.49s/it]


In [ ]:
query = make_prompt(questions[15]) + '\n返回json。'
print(query)

query = make_prompt(questions[44]) + '\n返回json。'
print(query)

# Compare Results

Compare the stage 1 results => find the differences => get the correct answer.

In [88]:
import os
import json

# Path to the folder containing model answer files
dir_path = os.path.join(cwd, 'answer_tmp')

# List of model names
models = ['deepseek_241206', 'glm_4_plus', 'claude_3_5_sonnet_20241022']

# Create a dictionary of file paths for each model's JSON file
model_files = {model: os.path.join(dir_path, f"stage_1-{model}-answer.json") for model in models}

# Dictionary to store the data of each model
model_data = {}

# Read the JSON data for each model
for model, file_path in model_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        model_data[model] = json.load(f)

# Dictionary to store the data_source for each question id across different models
data_sources = {}

# Traverse through each model's data to extract the data_source for each question id
for model in models:
    for entry in model_data[model]:
        for team in entry['team']:
            # Extract the id and data_source and store them in the data_sources dictionary
            ###
            # Handle deepseek output format issue, it appends an useless whitespace in the "id"
            ###
            data_sources.setdefault(team['id'].replace(' ', ''), {}).update({model: team['data_source']})
            

# Compare the data_source for each question id across models
for question_id, sources in data_sources.items():
    # Check if the data_source is consistent across models
    # Convert each model's data_source to a JSON string (to handle the dictionary comparison)
    serialized_sources = {model: json.dumps(ds, sort_keys=True) for model, ds in sources.items()}
    
    # If there are any differences in data_source, output the details
    if len(set(serialized_sources.values())) > 1:
        print(f"Question ID: {question_id}")
        
        # Calculate the maximum length of model names to align the output
        max_model_length = max(len(model) for model in models)
        
        # Print the data_source for each model, with aligned output
        for model in models:
            # Left-align model names with the calculated maximum length
            print(f"{model.ljust(max_model_length)}: {serialized_sources.get(model, 'No data available')}")
        
        print("\n")


Question ID: tttt----4----4-2-3
deepseek_241206           : [{"database": "AStockIndustryDB", "table": "LC_COConcept"}, {"database": "ConstantDB", "table": "SecuMain"}]
glm_4_plus                : [{"database": "AStockIndustryDB", "table": "LC_COConcept"}]
claude_3_5_sonnet_20241022: [{"database": "AStockIndustryDB", "table": "LC_COConcept"}, {"database": "ConstantDB", "table": "SecuMain"}]


Question ID: tttt----5----5-2-1
deepseek_241206           : [{"database": "AStockIndustryDB", "table": "LC_ConceptList"}]
glm_4_plus                : [{"database": "AStockIndustryDB", "table": "LC_COConcept"}]
claude_3_5_sonnet_20241022: [{"database": "AStockIndustryDB", "table": "LC_ConceptList"}]


Question ID: tttt----5----5-2-2
deepseek_241206           : [{"database": "AStockIndustryDB", "table": "LC_ConceptList"}]
glm_4_plus                : [{"database": "AStockIndustryDB", "table": "LC_COConcept"}]
claude_3_5_sonnet_20241022: [{"database": "AStockIndustryDB", "table": "LC_ConceptList"}]


